## Data exploration

In [2]:
import pandas as pd

news_df = pd.read_csv("Sora_LREC2020_biasedsentences.csv")

Seems that each instance consists of an article with sentences and a bias score between 1 and 4 is assigned to the article as a whole, the title, and each sentence.

In [28]:
news_test_sentence = news_df.iloc[0] # visual inspection
print(f"sentence: {news_test_sentence['s0']}", f"score: {news_test_sentence["0"]}")

print(f"dataset score range: {news_df["article_bias"].min(),news_df["article_bias"].max()}")

sentence: [0]: LOUISVILLE - Dan Johnson posted a final message on Facebook to his friends and family on Wednesday afternoon. score: 3
dataset score range: (np.int64(1), np.int64(4))


## Baseline
Takes ~10 mins to run- results is MSE = 0.04640446037612758

In [4]:
# source: https://huggingface.co/d4data/bias-detection-model
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("d4data/bias-detection-model")
model = TFAutoModelForSequenceClassification.from_pretrained("d4data/bias-detection-model")

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer) # cuda = 0,1 based on gpu availability

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at d4data/bias-detection-model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [43]:
def rescale(score):
    '''baseline classifier gives scores in the range of 0.5 to 1 (i think)
        making a linear function to map the score to the correct range
    '''
    current_min_score = 1
    current_max_score = 4
    target_min_score = 0.5
    target_max_score = 1
    m = (target_max_score-target_min_score)/(current_max_score-current_min_score)
    b = target_min_score-(m*current_min_score)
    
    return (m*score)+b

# comparing bias score obtained from classifier to true bias score for a test sentence from the news dataset
test_sent_pred = classifier(news_test_sentence["s0"])
test_sent_y = rescale(news_test_sentence["0"])
print(f"pred: {test_pred[-1]['score']}, y: {test_sent_y}")

pred: 0.762514591217041, y: 0.8333333333333334


In [ ]:
test = False
def compare_predictions(df, regressor):
    '''
    compute mean squared error for all individual sentences and titles in the news dataset
    '''
    squared_error = 0
    sentence_count = 0

    # iterate through every instance (article)
    for article_index, row in df.iterrows():
        # if test and article_index <30:
        #     continue
        # elif test and article_index > 40:
        #     break
        print(article_index)

        # iterate through every sentence
        for i in range(0, 20):  # Sentences are named f"s0" to f"s19"
            sentence = row[f"s{i}"]
            label = row[f"{i}"]  # The label column for the ith sentence
            # compute bias score with regressor
            try:
                prediction_list = regressor(sentence)
                pred = prediction_list[-1]['score']
            except ValueError: # if the sentence is a NaN, change the prediction to minimum bias value
                pred = 0.5
            y = rescale(label)
            # compute square error as the square of the difference between the predicted and label
            squared_error += (y-pred)**2
            sentence_count += 1

        # Handle the title
        title = row["doctitle"]
        label = row["t"]
        prediction_list = regressor(title)
        pred = prediction_list[-1]['score']
        y = rescale(label)
        squared_error += (y-pred)**2
        sentence_count += 1

    # divide the total square erorr by the total number of sentences (keep track with counter) to obtain MSE
    return squared_error/sentence_count

compare_predictions(news_df, classifier)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214


0.04640446037612758

## Fine tune classifier and compare to baseline

In [ ]:
#preprocess news data
def extract_sentences(df):
    sentences_list = []
    for _, row in df.iterrows():
        for i in range(0, 20):  # Sentences are named f"s0" to f"s19"
            

In [ ]:
...
from transformers import TrainingArguments
from transformers import Trainer

args = TrainingArguments(
    "baseline-bias-regressor",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,
    num_train_epochs = 3,
    weight_decay = 0.01,
)

trainer = Trainer(
    model = model,
    args = args,
    
)

## Make predictions for X notes data

In [ ]:
...

## Compare classifier predictions to human predictions

In [ ]:
# my idea for how to evaluate:
# 1) we set some guidelines for how we will evaluate notes 
# 2) we individually make labels for one days worth of notes (that's about 200 notes; this would be the train set)
# 3) we calculate out interrater agreement (if it's very bad, we revise our guidelines and re-label)
# 4) we average our labels and use that to further fine-tune the classifier
# 5) we repeat step 2 on a new set of notes (this would be the test set)
# 6) we make predictions using the classifier obtained from step 4
# 7) either we calculate the MSE using our labels as ground truth, or we calculate three interrater agreements (chico-andrew, chico-classifier, andrew-classifier)
# and see if the human-human agreement is better than the human-classifier agreement.
...